# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [59]:
import re
from collections import Counter
import pandas as pd
import numpy as np

def get_words(text): return re.findall(r'\w+', text.lower())

file = open('big.txt').read()
WORDS = Counter(get_words(file))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]).union(known(edits1(word))) or known(edits2(word)) or [word])

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

In [60]:
from typing import defaultdict

bigrams = defaultdict(int)
trigrams = defaultdict(int)
fourgrams = defaultdict(int)
fivegrams = defaultdict(int)

In [61]:
with open('w2_.txt', 'r') as file:  # Open the file 'w2_.txt' for reading
    lines = file.read().splitlines()  # Read the lines of the file and split them
    for line in lines:  # Iterate through each line in the file
        line = line.strip().split('\t')
        
        frequency = int(line[0])  # Extract the frequency value from the first element of the split line
        
        bigram = tuple(line[1:])  # Convert the rest of the line into a tuple, representing a bigram
        
        bigrams[bigram] += frequency  # Update the count of the bigram in the 'bigrams' dictionary


In [62]:
with open('w5_.txt', 'r') as file:  # Open the file 'w5_.txt' for reading
    lines = file.read().splitlines()  # Read the lines of the file and split them
    for line in lines:  # Iterate through each line in the file
        line = line.strip().split('\t')  # Remove leading/trailing whitespaces and split the line by tab character
        
        frequency = int(line[0])  # Extract the frequency value from the first element of the split line
        
        fivegram = tuple(line[1:])  # Convert the rest of the line into a tuple, representing a fivegram
        fourgram = [fivegram[:4], fivegram[1:]]  # Create two fourgrams from the fivegram
        trigram = [fivegram[:3], fivegram[1:4], fivegram[2:]]  # Create three trigrams from the fivegram
        
        if fivegram not in fivegrams:  # Check if the fivegram is not already in the 'fivegrams' dictionary
            fivegrams[fivegram] = frequency  # Add the fivegram to the 'fivegrams' dictionary with its frequency
            
        for t in trigram:  # Iterate through each trigram
            trigrams[t] += frequency  # Increment the count of the trigram in the 'trigrams' dictionary
            
        for t in fourgram:  # Iterate through each fourgram
            fourgrams[t] += frequency  # Increment the count of the fourgram in the 'fourgrams' dictionary


In [63]:
def find_word(word, context):
    # Limit the context to the last four words
    context = context[-4:]
    
    freq_context = []
    for i in range(1, 5):
        cur_context = context[-i:]
        
        # Check if the subsentence exists in n-grams dictionaries and add its frequency to the list
        subsentence = (*cur_context, word)
        if i == 1 and subsentence in bigrams:
            frequency = bigrams[subsentence]
            freq_context.append((i, frequency))
        elif i == 2 and subsentence in trigrams:
            frequency = trigrams[subsentence]
            freq_context.append((i, frequency))
        elif i == 3 and subsentence in fourgrams:
            frequency = fourgrams[subsentence]
            freq_context.append((i, frequency))
        elif i == 4 and subsentence in fivegrams:
            frequency = fivegrams[subsentence]
            freq_context.append((i, frequency))
        
    return freq_context
            
            
        

In [64]:
find_word("woods", ["a", "babe", "in", "the"])

[(1, 8805), (2, 630), (3, 16), (4, 16)]

In [65]:
def find_max_context(contexts):
    # Initialize variables to store the maximum length, maximum frequency, and index of the maximum context
    max_len = 0
    max_freq = 0
    idx = -1
    
    # Iterate through each context in the list of contexts
    for i in range(len(contexts)):
        # Check if the length of the current context is greater than the maximum length found so far
        if len(contexts[i]) > max_len:
            # Update the maximum length, maximum frequency, and index of the maximum context
            max_len = len(contexts[i])
            max_freq = sum(c for _, c in contexts[i])  # Sum up the frequencies of all elements in the context
            idx = i
        # If the lengths are equal, compare the total frequencies
        elif len(contexts[i]) == max_len:
            # Calculate the total frequency of the current context
            context_freq = sum(c for _, c in contexts[i])
            # If the total frequency of the current context is greater than the maximum frequency found so far
            if context_freq > max_freq:
                # Update the maximum frequency and index of the maximum context
                max_freq = context_freq
                idx = i
                
    # Return the index of the context with the maximum length and highest total frequency
    return idx


In [66]:
def correct(word, context):
    # Generate a list of candidate corrections for the misspelled word
    candids = list(candidates(word))
    
    # Initialize lists to store new context with non-empty corrections and their corresponding indices
    new_context = []
    non_empty = []
    
    # Iterate through each candidate correction and its index
    for i, cand in enumerate(candids):
        # Find the corrected word corresponding to the candidate in the context
        new_word = find_word(cand, context)
        # If the corrected word is not empty, add it to the new context and store its index
        if new_word != []:
            new_context.append(new_word)
            non_empty.append(i)
    
    # Find the index of the context with the maximum length and highest total frequency
    idx = find_max_context(new_context)
    
    # If there are non-empty corrections available
    if non_empty != []:
        # Retrieve the corrected word corresponding to the index
        correct_word = candids[non_empty[idx]]
    else:
        # If no non-empty corrections are found, use a default correction for the misspelled word
        correct_word = correction(word)
    
    # Return the corrected word
    return correct_word


In [67]:
test_text_with_mistakes = \
""""
I red the book yesturday and it was really interessing.
He's alwais late for meetings, it's so frustraiting.
She baught a new dress for the occassion, it looks amasing on her.
We had a deliscious dinner last night, with lot's of different dishes.
Their cat is so cuite, with it's fluffy fur and big green eyes.
I can't belive how bueatiful the sunset was yesterday.
He's been working so hard latley, I hope he gets some rest soon.
She's such a gr8 friend, always there when I need her.
I herd the news about the accident, it was truely shoking.
We're going too the beach this weakend, I can't wait to relax in the sun.
"""

test_text_corrected = \
"""
I read the book yesterday and it was really interesting.
He's always late for meetings, it's so frustrating.
She bought a new dress for the occasion, it looks amazing on her.
We had a delicious dinner last night, with lots of different dishes.
Their cat is so cute, with its fluffy fur and big green eyes.
I can't believe how beautiful the sunset was yesterday.
He's been working so hard lately, I hope he gets some rest soon.
She's such a great friend, always there when I need her.
I heard the news about the accident, it was truly shocking.
We're going to the beach this weekend, I can't wait to relax in the sun.
"""

In [68]:
for line in test_text_with_mistakes.splitlines():
    words = get_words(line)
    for i in range(len(words)):
        if words[i] not in WORDS:
            incorrect_word = words[i]
            corrected_word = correct(words[i], words[:i])
            words[i] = corrected_word
            print(f"{incorrect_word} -> {corrected_word}")

yesturday -> yesterday
interessing -> interesting
alwais -> always


frustraiting -> frustraiting
baught -> bought
occassion -> occasion
amasing -> amazing
deliscious -> delicious
cuite -> quite
belive -> believe
bueatiful -> beautiful
latley -> lately
gr8 -> grm
truely -> truly
shoking -> showing
weakend -> weekend


In [69]:
def correct_mistakes(context):
    # Iterate through each line in the context
    for line in context.splitlines():
        # Tokenize the line into words
        words = get_words(line)
        
        # Iterate through each word in the line
        for i in range(len(words)):
            # Check if the word is not found in the pre-defined WORDS set (possibly a misspelling)
            if words[i] not in WORDS:
                # Correct the misspelled word based on the context before it
                corrected_word = correct(words[i], words[:i])
                # Replace the original word with the corrected one
                words[i] = corrected_word
                
    # Reconstruct the context with corrected words and return it
    return context


In [70]:
print(correct_mistakes(test_text_with_mistakes))

"
I red the book yesturday and it was really interessing.
He's alwais late for meetings, it's so frustraiting.
She baught a new dress for the occassion, it looks amasing on her.
We had a deliscious dinner last night, with lot's of different dishes.
Their cat is so cuite, with it's fluffy fur and big green eyes.
I can't belive how bueatiful the sunset was yesterday.
He's been working so hard latley, I hope he gets some rest soon.
She's such a gr8 friend, always there when I need her.
I herd the news about the accident, it was truely shoking.
We're going too the beach this weakend, I can't wait to relax in the sun.



## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

I started with exploring the dataset. There were 3 provided datasets: big.txt, w2_.txt, w5_.txt. The first one consists of combined excerpts from public domain books sourced from Project Gutenberg, along with lists of the most commonly used words from Wiktionary and the British National Corpus, merged together. The second one contains bigrams, I decided to use them as they are. The third file consists of 5-grams. I decided to divide it into 3-grams, 4-grams and 5-grams. Using this method I can get more information about the dependencies in the text. Using n more than 5 in n-grams is not useful for me, because of lack of computational power. Therefore, I limited context to last four words. One of the biggest limitation of provided datasets is lack of knowledge where n-gram is located. There were no tokens such as \<s> - start of sentence or <\s> - end of sentence. Therefore we do not know location of the ngram and we cannot use this information for prediction. We assume that n-gram located somewhere in the center. However such a rough guess still got good results.

I assumed that words that appear in a greater number of different contexts should be used more frequently. Once contexts are identified for all potential corrections, the algorithm picks the correction that appears in the context with the longest length and highest frequency. This approach is designed to give precedence to corrections backed by larger and more commonly occurring contexts, thereby enhancing the accuracy of spell correction.

According to [1], there is no need to correct words with length less than 3. It does not make sense and hence computing suggestions and ranking them is
not logical.

For testing part, I synthesized dataset using edits1 and edits2 functions of Norving's solution. I assume that user misspelled a word 1-2 times. Also I assume that 1 type of error (edits1) is as possible as 2 type of error (edits2). Moreover, for simplicity, I synthesized mistakes only in the last word in the sentence. It is quite significant limitation, but for now it gives a good undestanding of model performance and gives impetus to the development of this method. Synthesizing error only in the last step guarantees that word has context.

References: \
[1] - https://assets.amazon.science/81/43/a1eaa3814b009481d306deb69b02/scipub-1158.pdf



## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [71]:
with open('imdb_labelled.txt', 'r') as f:
    test = [line.split('\t')[0].strip() for line in f][:700]

In [72]:
import random

def make_mistake(words):
    # Randomly choose between two types of mistakes: edits1 or edits2
    choice = random.randint(0, 1)
    
    # Generate incorrect words based on the chosen type of mistake
    if choice == 0:
        incorrect_words = edits1(words[-1])
    elif choice == 1:
        incorrect_words = edits2(words[-1])
    
    # Randomly select one incorrect word from the generated set of incorrect words
    incorrect_word = random.choice(list(incorrect_words))
    
    # Replace the last word in the original list with the selected incorrect word
    words[-1] = incorrect_word
    
    # Form an incorrect sentence by joining the modified list of words
    incorrect_sentence = " ".join(words)
    
    # Return the incorrect sentence along with the selected incorrect word
    return incorrect_sentence, incorrect_word


In [73]:
make_mistake(get_words("I love football"))

('i love lootball', 'lootball')

### Test my solution

In [74]:
count = 0
for sentence in test:
    words = get_words(sentence)
    correct_word = words[-1]
    incorrect_sentence, incorrect_word = make_mistake(words)
    
    corrected_word = correct(incorrect_word, words[:-1])
    if corrected_word == correct_word:
        count += 1

print(count / len(test))

0.7385714285714285


### Test Norvig's solution

In [75]:
count = 0
for sentence in test:
    words = get_words(sentence)
    correct_word = words[-1]
    incorrect_sentence, incorrect_word = make_mistake(words)
    
    corrected_word = correction(incorrect_word)
    if corrected_word == correct_word:
        count += 1

print(count / len(test))

0.6785714285714286


According to the results, my solution is 7% better than Norvig's solution, because my solution uses context in contrast to Norvig's solution.